# Blinkit Sales Analytics – Feature Engineering

## Objective
Transform cleaned and enriched datasets into feature-rich tables suitable
for modeling and dashboarding.

Focus Areas:
- Customer-level features
- Product-level features
- Delivery performance features
- Final modeling dataset creation


In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)


In [2]:
orders = pd.read_csv("../data/interim/orders_clean.csv")
order_items = pd.read_csv("../data/interim/order_items_clean.csv")
products = pd.read_csv("../data/interim/products_clean.csv")
delivery = pd.read_csv("../data/interim/delivery_clean.csv")

orders.shape, order_items.shape, products.shape, delivery.shape


((5000, 10), (5000, 4), (268, 10), (5000, 8))

In [3]:
base_df = (
    order_items
    .merge(orders, on="order_id", how="left")
    .merge(products, on="product_id", how="left")
    .merge(
        delivery[
            ["order_id", "delivery_time_minutes", "distance_km", "delivery_status"]
        ],
        on="order_id",
        how="left"
    )
)

base_df.shape


(5000, 25)

In [4]:
# order_total already exists and is final billed value
# DO NOT create order_value
base_df["order_total"].describe()


count    5000.00000
mean     2201.86170
std      1303.02438
min        13.25000
25%      1086.21500
50%      2100.69000
75%      3156.88250
max      6721.46000
Name: order_total, dtype: float64

In [5]:
base_df["order_date"] = pd.to_datetime(base_df["order_date"])

base_df["order_hour"] = base_df["order_date"].dt.hour
base_df["order_dayofweek"] = base_df["order_date"].dt.dayofweek
base_df["is_weekend"] = base_df["order_dayofweek"].isin([5, 6]).astype(int)


In [6]:
customer_features = (
    base_df.groupby("customer_id")
    .agg(
        customer_total_orders=("order_id", "nunique"),
        customer_total_spend=("order_total", "sum"),
        customer_avg_order_value=("order_total", "mean"),
        customer_total_quantity=("quantity", "sum"),
        last_order_date=("order_date", "max")
    )
    .reset_index()
)


In [7]:
max_date = base_df["order_date"].max()

customer_features["customer_recency_days"] = (
    max_date - customer_features["last_order_date"]
).dt.days

customer_features["is_repeat_customer"] = (
    customer_features["customer_total_orders"] > 1
).astype(int)


In [8]:
product_features = (
    base_df.groupby("product_id")
    .agg(
        product_name=("product_name", "first"),
        category=("category", "first"),
        total_units_sold=("quantity", "sum"),
        product_total_revenue=("order_total", "sum")
    )
    .reset_index()
)


In [9]:
total_revenue = product_features["product_total_revenue"].sum()

product_features["product_revenue_share"] = (
    product_features["product_total_revenue"] / total_revenue
)


In [10]:
delivery_features = (
    delivery.groupby("order_id")
    .agg(
        delivery_time_minutes=("delivery_time_minutes", "mean"),
        distance_km=("distance_km", "mean")
    )
    .reset_index()
)

delivery_features["delayed_flag"] = (
    delivery_features["delivery_time_minutes"] > 30
).astype(int)


In [11]:
final_features = (
    base_df
    .merge(customer_features, on="customer_id", how="left")
    .merge(product_features, on="product_id", how="left")
    .merge(delivery_features, on="order_id", how="left")
)


In [ ]:
columns_to_drop = [
      # intermediate only
]

final_features.drop(columns=columns_to_drop, inplace=True, errors="ignore")


In [12]:
final_features.to_csv(
    "../data/processed/final_features_clean.csv",
    index=False
)

final_features.shape


(5000, 43)